In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

In [32]:
base_dir = 'images/split/training'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

data_gen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.5
)

train_generator = data_gen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale',
    subset='training'
)

validation_generator = data_gen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale',
    subset='validation'
)

Found 2850 images belonging to 5 classes.
Found 2850 images belonging to 5 classes.


In [33]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 1)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(512, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    # Dropout(0.5),
    Dense(5, activation='softmax')
])


In [34]:
model.compile(
    optimizer=Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [35]:
from keras.callbacks import Callback

class BaselineEarlyStopping(EarlyStopping):
    def __init__(self, baseline, **kwargs):
        super(BaselineEarlyStopping, self).__init__(**kwargs)
        self.baseline = baseline

    def on_epoch_end(self, epoch, logs=None):
        current = logs.get(self.monitor)
        if current is None:
            return

        # Only stop training if the baseline is reached
        if current >= self.baseline:
            super(BaselineEarlyStopping, self).on_epoch_end(epoch, logs)

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=10,
    verbose=1,
    mode='max',
    restore_best_weights=True
)

In [36]:
history = model.fit(
    train_generator,
    epochs=999999999,
    validation_data=validation_generator,
    verbose=1,
    callbacks=[early_stopping]
)

Epoch 1/999999999
90/90 [==============================] - 8s 91ms/step - loss: 1.5548 - accuracy: 0.2709 - val_loss: 1.5225 - val_accuracy: 0.3330
Epoch 2/999999999
90/90 [==============================] - 8s 85ms/step - loss: 1.2438 - accuracy: 0.4905 - val_loss: 1.2496 - val_accuracy: 0.5175
Epoch 3/999999999
90/90 [==============================] - 8s 84ms/step - loss: 0.8781 - accuracy: 0.6635 - val_loss: 1.0264 - val_accuracy: 0.5811
Epoch 4/999999999
90/90 [==============================] - 8s 85ms/step - loss: 0.7248 - accuracy: 0.7256 - val_loss: 0.9618 - val_accuracy: 0.6484
Epoch 5/999999999
90/90 [==============================] - 8s 86ms/step - loss: 0.4674 - accuracy: 0.8242 - val_loss: 0.8840 - val_accuracy: 0.6853
Epoch 6/999999999
90/90 [==============================] - 8s 87ms/step - loss: 0.3383 - accuracy: 0.8758 - val_loss: 0.5404 - val_accuracy: 0.8161
Epoch 7/999999999
90/90 [==============================] - 8s 87ms/step - loss: 0.3237 - accuracy: 0.8933 - val_

In [37]:
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation accuracy: {accuracy * 100:.2f}%')

90/90 [==============================] - 8s 92ms/step - loss: 0.3402 - accuracy: 0.9018
Validation accuracy: 90.18%


In [38]:
# def predict_uploaded_image(model, img_path):
#     img = image.load_img(img_path, target_size=(150, 150))
#     imgplot = plt.imshow(img)
#     plt.show()

#     img_array = image.img_to_array(img)
#     img_array = np.expand_dims(img_array, axis=0) / 255.

#     predictions = model.predict(img_array)
#     classes = ['paper', 'rock', 'scissors']
#     predicted_class = classes[np.argmax(predictions)]

#     return predicted_class

# uploaded = files.upload()

# for fn in uploaded.keys():
#     predicted_class = predict_uploaded_image(model, fn)
#     print(f'Gambar {fn} diprediksi sebagai: {predicted_class}')

In [39]:
img_path = "../images/split/testing/Colin_Powell/image_3.png"

img = image.load_img(img_path, target_size=(150, 150), color_mode='grayscale')
imgplot = plt.imshow(img)
plt.show()

img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.

predictions = model.predict(img_array)
classes = ['Colin', 'Donald', 'George', 'Gerhard', 'Tony']
predicted_class = classes[np.argmax(predictions)]


print(f'Gambar diprediksi sebagai: {predicted_class}')

FileNotFoundError: [Errno 2] No such file or directory: '../images/split/testing/Colin_Powell/image_3.png'